In [1]:
import pandas as pd
import json

Inception v3 trained on balanced dataset, focal loss $\gamma = 2$, 5 epochs, 100 batch size, Adam lr=0.0005

In [2]:
# path = "../results/inception_v3/I1 - fl/"
# path = "../results/inception_v3/I2 - xen/"
path = "../results/AlexNet/A1 - xen/"
# path = "../results/AlexNet/A1 - xen reg/"

In [3]:
df = pd.read_csv(path+"avg_metrics.csv")

In [4]:
df.columns

Index(['image_idxs', 'accuracy', 'precision', 'specificity', 'sensibility',
       'IoU', 'dice_coeff', 'f1_score'],
      dtype='object')

In [5]:
# Function to parse indices string into list of ints
def parse_json_column(json_str):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return None

# Apply the parsing function to the 'data' column
df["image_idxs"] = df["image_idxs"].apply(parse_json_column)

In [6]:
df

,image_idxs,accuracy,precision,specificity,sensibility,IoU,dice_coeff,f1_score
0,"[0, 1, 2]",0.924729,0.936870,0.771766,0.965847,0.906932,0.951105,0.951138
1,"[3, 4, 5]",0.813175,0.960840,0.900528,0.780989,0.756859,0.861599,0.861629
2,"[6, 7, 8]",0.960370,0.963044,0.439845,0.994812,0.958276,0.978643,0.978670
3,"[9, 10, 11]",0.930805,0.937141,0.646618,0.983894,0.922994,0.959923,0.959949
4,"[12, 13, 14]",0.943625,0.945754,0.428313,0.994108,0.940493,0.969233,0.969328
5,"[15, 16, 17]",0.883235,0.929338,0.765537,0.888965,0.834407,0.907891,0.908703
6,"[18, 19, 20]",0.950000,0.954426,0.618978,0.990647,0.945974,0.972161,0.972199
7,"[21, 22, 23]",0.876968,0.787218,0.558669,0.974598,0.842288,0.910503,0.870943
8,"[24, 25, 26]",0.926233,0.768749,0.450091,0.990314,0.913857,0.951723,0.865578
9,"[27, 28]",0.916343,0.916406,0.503293,0.988319,0.906585,0.950987,0.951005


In [7]:
metrics_df = df.drop(columns=['image_idxs'])

In [8]:
cols = metrics_df.columns
n_samples, n_features = metrics_df.shape

In [9]:
list(cols)

['accuracy',
 'precision',
 'specificity',
 'sensibility',
 'IoU',
 'dice_coeff',
 'f1_score']

In [10]:
means = metrics_df.mean()
means

accuracy       0.912548
precision      0.909979
specificity    0.608364
sensibility    0.955249
IoU            0.892866
dice_coeff     0.941377
f1_score       0.928914
dtype: float64

In [11]:
non_nan_counts = metrics_df.count()
stds = metrics_df.std()/non_nan_counts
stds

accuracy       0.004389
precision      0.007114
specificity    0.016252
sensibility    0.006890
IoU            0.006287
dice_coeff     0.003680
f1_score       0.004742
dtype: float64

In [12]:
n_decimals = 4
cross_val_data = [f"{means.iloc[i]:0.{n_decimals}f} +- {stds.iloc[i]:0.{n_decimals}f}" for i in range(n_features)]

In [13]:
cross_val_metrics_df = pd.DataFrame([cross_val_data], columns=tuple(cols))
cross_val_metrics_df.loc[0]

accuracy       0.9125 +- 0.0044
precision      0.9100 +- 0.0071
specificity    0.6084 +- 0.0163
sensibility    0.9552 +- 0.0069
IoU            0.8929 +- 0.0063
dice_coeff     0.9414 +- 0.0037
f1_score       0.9289 +- 0.0047
Name: 0, dtype: object